In [15]:
# Implementasi dengan Algoritma KNN (scikit-learn)

In [26]:
import numpy as nm  
import matplotlib.pyplot as mtp  
import pandas as pd
from sklearn.model_selection import train_test_split 
  
dataset = pd.read_csv('data_train.csv')  
x = dataset.iloc[:, [2, 3]].values  
y = dataset.iloc[:, 4].values
    
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [27]:
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)  

In [31]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(x_train, y_train)

KNeighborsClassifier()

In [32]:
y_pred = classifier.predict(x_test)  

In [30]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.1657142857142857


*****